In [47]:
%run ../../Utils/yp_utils.py

# Initial setup

In [48]:
paper_pmid = 20337531
paper_name = 'dias_sa_correia_2010' 

In [49]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [50]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [51]:
original_data1 = pd.read_csv('raw_data/TableS1.txt', header=None, names=['genes','orfs'], sep='\t')
original_data2 = pd.read_csv('raw_data/TableS2.txt', header=None, names=['genes','orfs'], sep='\t')

In [52]:
print('Original data dimensions: %d x %d' % (original_data1.shape))
print('Original data dimensions: %d x %d' % (original_data2.shape))

Original data dimensions: 288 x 2
Original data dimensions: 453 x 2


In [53]:
original_data1['orfs'] = original_data1['orfs'].astype(str)
original_data2['orfs'] = original_data2['orfs'].astype(str)

In [54]:
# Eliminate all white spaces & capitalize
original_data1['orfs'] = clean_orf(original_data1['orfs'])
original_data2['orfs'] = clean_orf(original_data2['orfs'])

In [55]:
# Translate to ORFs 
original_data1['orfs'] = translate_sc(original_data1['orfs'], to='orf')
original_data2['orfs'] = translate_sc(original_data2['orfs'], to='orf')

In [56]:
# Make sure everything translated ok
t = looks_like_orf(original_data1['orfs'])
print(original_data1.loc[~t,])

Empty DataFrame
Columns: [genes, orfs]
Index: []


In [57]:
# Make sure everything translated ok
t = looks_like_orf(original_data2['orfs'])
print(original_data2.loc[~t,])

Empty DataFrame
Columns: [genes, orfs]
Index: []


In [58]:
original_data1.set_index('orfs', inplace=True)
original_data2.set_index('orfs', inplace=True)
original_data1.index.name='orf'
original_data2.index.name='orf'

In [59]:
# High-susceptibility
original_data1['data'] = -2

# Moderate-susceptibility
original_data2['data'] = -1

In [60]:
original_data1 = original_data1[['data']].copy()
original_data2 = original_data2[['data']].copy()

In [61]:
original_data1 = original_data1.groupby(original_data1.index).mean()
original_data2 = original_data2.groupby(original_data2.index).mean()

In [62]:
original_data1.shape

(286, 1)

In [63]:
original_data2.shape

(453, 1)

In [64]:
original_data = original_data1.join(original_data2, lsuffix='_high', rsuffix='_mod', how='outer')

In [65]:
original_data['data'] = original_data[['data_high','data_mod']].mean(axis=1)

In [66]:
original_data.drop(columns=['data_high','data_mod'], inplace=True)

In [67]:
original_data.shape

(739, 1)

# Load & process tested strains

In [68]:
tested = pd.read_excel('raw_data/List of strains tested.xlsx', sheet_name='Tabelle2')

In [69]:
tested.head()

,ORF,slow growth?,Unnamed: 2
0,YAL068C,NaN,NaN
1,YAL067C,NaN,NaN
2,YAL066W,NaN,Note: when a strain had slow growth in control...
3,YAL065C,NaN,NaN
4,YAL062W,NaN,NaN


In [70]:
tested['orf'] = tested['ORF'].astype(str)

In [71]:
tested['orf'] = clean_orf(tested['orf'])

In [72]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [73]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,:])

Empty DataFrame
Columns: [ORF, slow growth?, Unnamed: 2, orf]
Index: []


In [74]:
tested_orfs = tested['orf'].unique()

In [75]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

['YBL014C',
 'YBR123C',
 'YBR132C',
 'YBR152W',
 'YBR193C',
 'YDL043C',
 'YDL126C',
 'YDR302W',
 'YER087W',
 'YER133W',
 'YER165W',
 'YFR029W',
 'YGL097W',
 'YHL015W',
 'YHR172W',
 'YIL021W',
 'YIL026C',
 'YIL031W',
 'YIL126W',
 'YIR011C',
 'YJL026W',
 'YJL086C',
 'YJL097W',
 'YJL111W',
 'YJR006W',
 'YKL033W',
 'YKL059C',
 'YKL152C',
 'YKL165C',
 'YLR197W',
 'YMR033W',
 'YMR038C',
 'YMR043W',
 'YMR061W',
 'YMR309C',
 'YNL039W',
 'YNL245C',
 'YNR017W',
 'YNR038W',
 'YNR043W',
 'YPL233W',
 'YPR187W']

In [76]:
# Remove missing (the data list contains HAP and HET screen results, so some of the genes are likely to be essential)
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

In [77]:
original_data.shape

(4932, 1)

# Prepare the final dataset

In [78]:
data = original_data.copy()

In [79]:
dataset_ids = [16607]
datasets = datasets.reindex(index=dataset_ids)

In [80]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [81]:
data.head()

dataset_id,16607
data_type,value
orf,
YAL068C,0.0
YAL067C,0.0
YAL066W,0.0
YAL065C,0.0
YAL062W,0.0


## Subset to the genes currently in SGD

In [82]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 20


In [83]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16607
,data_type,value
gene_id,orf,
1869,YAL068C,0.0
61,YAL067C,0.0
60,YAL066W,0.0
1727,YAL065C,0.0
57,YAL062W,0.0


# Normalize

In [84]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [85]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [86]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      16607       
data_type       value valuez
gene_id orf                 
1869    YAL068C   0.0    0.0
61      YAL067C   0.0    0.0
60      YAL066W   0.0    0.0
1727    YAL065C   0.0    0.0
57      YAL062W   0.0    0.0

# Print out

In [87]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [91]:
# from IO.save_data_to_db3 import *

In [92]:
# save_data_to_db(data_all, paper_pmid)